<a href="https://colab.research.google.com/github/Adrok24/classification-of-plant-diseases/blob/main/web_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from tensorflow.keras.models import load_model
from zipfile import ZipFile 
import os
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
#files.download('file.csv')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Web Scrapper

In [23]:
!pip install kora -q
from kora.selenium import wd


In [11]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [12]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [18]:
def search_and_download(search_term:str,wd,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

Instanciamos el web scrapper para probar, funciona bien pero a veces se baja imagenes de hojas enfermas entre las sanas o con patologias distintas.

In [35]:
import time
import requests, io, hashlib
from PIL import Image


search_and_download('tomato black dots leaf', wd, 'drive/My Drive/tomato_black_dots', number_images=10)

Found: 100 search results. Extracting links from 0:100
Found: 10 image links, done!
SUCCESS - saved https://bloximages.chicago2.vip.townnews.com/journalstar.com/content/tncms/assets/v3/editorial/b/e1/be16edc4-71d7-5ff3-be84-da417edd8b96/5592c2e0b6df8.image.jpg - as drive/My Drive/tomato_black_dots/tomato_black_dots_leaf/91ac18f67a.jpg
SUCCESS - saved https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F37%2F2020%2F04%2F22%2Fseptoria-spot-tomato-plant-ea2ab44d.jpg - as drive/My Drive/tomato_black_dots/tomato_black_dots_leaf/2d3a66d22a.jpg
SUCCESS - saved https://images.saferbrand.com/is/image/woodstream/sb_us_t_12_septor_Flickr-reuse-Scot-Nelson3?qlt=75 - as drive/My Drive/tomato_black_dots/tomato_black_dots_leaf/4c8a23e731.jpg
SUCCESS - saved https://www.gardeningknowhow.com/wp-content/uploads/2016/12/tomato-black-stem.jpg - as drive/My Drive/tomato_black_dots/tomato_black_dots_leaf/7e12729778.jpg
SUCCESS - saved https://apps.